In [1]:
!pip install datasets
!pip install torch torchvision torchaudio
!pip install rouge-score nltk
!pip install sentencepiece #что такое sentencepiece?
!pip install evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=af0d890d84aa864a1712ce1d1069554659f5c55870cf3deed3fd00b01e8500c3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.5 MB/s eta 0:00:00


Подробнее о sentencepiece: https://github.com/google/sentencepiece

In [37]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fedor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
import accelerate

accelerate.__version__

'0.25.0'

In [39]:
from datasets import load_dataset, load_metric
from transformers import (
    DataCollatorForSeq2Seq,
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    pipeline,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import evaluate
import numpy as np
import datasets

In [40]:
#узнайте подробнее о модели mT5
#в чем вообще особенность модели Т5?

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")  # cointegrated/rut5-small google/mt5-small
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

# tokenizer = MT5Tokenizer.from_pretrained("results/checkpoint-121500")  # это если мы хотим подгрузить веса модели которую уже обучали
# model = MT5ForConditionalGeneration.from_pretrained('results/checkpoint-121500')

In [41]:
max_target_tokens_count = 128
max_source_tokens_count = 512


def preprocess_function(examples):
    print(examples['text'][0])
    inputs = examples["text"]
    model_inputs = tokenizer(inputs, max_length=max_source_tokens_count, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=max_target_tokens_count, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [42]:
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0")
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|                                                                            | 0/60964 [00:00<?, ? examples/s]

Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, автомобилисты будут понимать, за что они платят, а дорожники будут иметь гарантированный доход. Кроме налога дорожные фонды будут пополняться за счет бюджетных средств и проезда по платным дорогам. Более того, транспортный налог предлагается завуалировать в акцизы на бензин. Привычную и раздражающую систему ежегодной оплаты квитанции предлагается изменить, включив налог в стоимость топлива. Минэкономразвития говорит об удвоении акцизы, которая сегодня составляет 3,32 

Map:   2%|█                                                               | 1000/60964 [00:02<02:25, 412.12 examples/s]

В США суд предъявил обвинение бывшей сотруднице инженерного центра General Motors в краже технологических секретов. По версии следствия, женщина-инженер и ее муж собирались продать документы по гибридным технологиям китайскому автопроизводителю Chery. Cуд предъявил 51-летней Саньшань Ду и ее мужу, 49-летнему Ю Циню, обвинение в преступном сговоре с целью продажи документов, содержащих коммерческую тайну, сообщает Associated Press. Расследование в отношении супругов продолжалось с 2006 года, всего им предъявлены обвинения по семи пунктам – Юй Цинь в том числе обвиняется в уничтожении улик. Подозреваемые отпущены судом под залог $10 000 c каждого с предписанием не покидать Детройта, пишет Detroit News. По версии следствия, Саньшань Ду, которая работала в одном из инженерных центров GM, похитила более 5000 страниц документов, касающихся разработок General Motors в области гибридных технологий. Супруги располагали крадеными секретными сведениями GM с 2003 по 2006 годы. При этом Ду, работая

Map:   3%|██                                                              | 2000/60964 [00:04<02:24, 406.93 examples/s]

...

Map:  79%|███████████████████████████████████████████████████              | 5000/6369 [00:13<00:03, 370.35 examples/s]

Вашингтон занимает деструктивную позицию в отношении Договора о всеобъемлющем запрещении ядерных испытаний (ДВЗЯИ) и готовит почву для возможного возобновления ядерных испытаний, говорится в заявлении МИД РФ. Оно было распространено по случаю 20-летия со дня ратификации ДВЗЯИ Москвой. «Этот ответственный шаг был продиктован решительным стремлением России двигаться по пути ядерного разоружения, укрепления режима нераспространения ядерного оружия и международной безопасности», — подчеркнули в сообщении МИД. В дипведомстве напомнили, что документ все еще не вступил в силу, поскольку восемь стран до сих пор не подписали или не ратифицировали договор. «Наиболее деструктивную позицию в этом отношении занимают США, которые официально объявили, что не намерены ратифицировать ДВЗЯИ, и готовят почву для возможного возобновления ядерных испытаний», — обратили внимание в российском внешнеполитическом ведомстве. Москва призвала государства, от которых зависит вступление в силу договора и которые ещ

Map:  94%|█████████████████████████████████████████████████████████████▏   | 6000/6369 [00:16<00:00, 370.44 examples/s]

Президент США Дональд Трамп не исключает санкций в отношении других китайских компаний после того, как американский лидер выпустил указы, фактически запрещающие приложения TikTok и WeChat на территории страны. Об этом сообщает Reuters. Во время пресс-конференции один из журналистов спросил Трампа, планирует ли он вводить санкции против других китайских компаний, например, Alibaba. «Мы рассматриваем эту возможность, да», — заявил американский президент. Alibaba Group является одной из крупнейших китайских корпораций. Ее основным видом деятельности является интернет-коммерция. Группе принадлежат международные интернет-площадки Alibaba.com, AliExpress.com, Taobao.com и Tmall.com. В начале августа Дональд Трамп подписал указ, который получил название «О борьбе с угрозой TikTok». Документ размещен на официальном сайте Белого дома. Согласно указу, американским компаниям отводится 45 дней на прекращение любых контактов с китайской компанией ByteDance, владеющей TikTok, в попытке «устранить уг

Map: 100%|█████████████████████████████████████████████████████████████████| 6369/6369 [00:17<00:00, 369.82 examples/s]


In [43]:
tokenized_dataset_train = tokenized_dataset['train'].select(range(3000))
tokenized_dataset_test= tokenized_dataset['test'].select(range(1000))

In [44]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model) #что такое DataCollatorForSeq2Seq?

Подробнее о метриках: https://clementbm.github.io/theory/2021/12/23/rouge-bleu-scores.html

In [45]:
rouge = load_metric("rouge") #Recall-Oriented Understudy for Gisting Evaluation
bleu = evaluate.load("bleu") #Bilingual Evaluation Understudy

In [46]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = {}
    result_rouge = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result.update({key: value.mid.fmeasure * 100 for key, value in result_rouge.items()})

    result_bleu = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results
    result["bleu"] = result_bleu["bleu"] * 100

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    result["char_len"] = np.mean([len(t) for t in decoded_preds])

    return {k: round(v, 4) for k, v in result.items()}

In [47]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/results",
    evaluation_strategy="steps",
    eval_steps=25,
    logging_steps=25,
    learning_rate=4e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=False,
    predict_with_generate=True,
    generation_max_length=20,
    generation_num_beams=5
)

In [48]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [49]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Gen Len,Char Len
25,6.856600,2.894139,4.929600,1.473500,4.874900,4.895600,0.395700,16.437400,63.176900
50,3.854500,2.739077,5.827200,1.672300,5.753000,5.781200,0.580000,18.663000,75.985900
75,3.514400,2.631428,6.997000,1.892800,6.899800,6.923700,0.367300,18.932300,73.254200
100,3.357000,2.587784,8.510700,2.381800,8.404000,8.437600,0.176000,18.998500,69.192400
125,3.291600,2.533018,8.653400,2.364200,8.541900,8.575100,0.126600,18.999700,68.443100
150,3.227300,2.527531,8.726300,2.265800,8.591300,8.655500,0.129700,19.000000,69.507400
175,3.179900,2.510088,8.578000,2.131300,8.476500,8.518400,0.131700,19.000000,69.531300
200,3.163100,2.503935,8.576100,2.131000,8.458600,8.505500,0.131700,19.000000,69.358500
225,3.151500,2.496379,8.604800,2.205400,8.485900,8.527700,0.132200,19.000000,69.280100


TrainOutput(global_step=238, training_loss=3.7013909315862574, metrics={'train_runtime': 5024.2318, 'train_samples_per_second': 12.134, 'train_steps_per_second': 0.047, 'total_flos': 3.221566116593664e+16, 'train_loss': 3.7013909315862574, 'epoch': 1.0})

In [50]:
trainer.evaluate()

{'eval_loss': 2.491903305053711,
 'eval_rouge1': 8.655,
 'eval_rouge2': 2.2025,
 'eval_rougeL': 8.5163,
 'eval_rougeLsum': 8.5717,
 'eval_bleu': 0.1337,
 'eval_gen_len': 19.0,
 'eval_char_len': 69.3521,
 'eval_runtime': 395.38,
 'eval_samples_per_second': 17.181,
 'eval_steps_per_second': 4.297,
 'epoch': 1.0}

In [75]:
#### Summarized texts can be obtained using the SummarizationPipeline
from transformers import SummarizationPipeline
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to('cpu')
summarizer = SummarizationPipeline(model=model, tokenizer=tokenizer)
text = "Black Midi — группа выпускников престижной арт-школы для талантливых подростков BRIT School, которая с первых синглов поразила всех свободой музыкального мышления. В их музыке много мат-рока, краут-рока (недаром они выступали с легендарным экс-вокалистом Can Дамо Судзуки), шума, атональной импровизации, негармоничных созвучий и внезапной смены музыкальных размеров. При этом результат получается крайне захватывающим: сложности и навороты, которые сами по себе звучат довольно отталкивающе, в их песнях оказываются на своем месте и не кажутся излишними усложнениями.  В 2018 году Black Midi заметил британский продюсер Дэн Кэри — группа поездила по миру с турами, выступила на важных фестивалях и показала себя как впечатляющий проект с большим будущим, она явно должна была полюбиться сразу нескольким поколениям поклонников андеграундной музыки. Дебютный альбом Black Midi «Schlagenheim» вышел в начале 2019 года и действительно поразил не только многих музыкальных критиков, но и слушателей: группа даже пробилась в британские чарты, что совсем нехарактерно для такой абразивной и не слишком дружелюбной музыки, не нацеленной на стадионы.  "
summary = summarizer(text, max_length=70, min_length=60, do_sample=False)

print(summary[0]['summary_text'])


В британской арт-школе Black Midi группа выпускников престижной арт-школы для талантливых подростков BRIT School, которая стала одним из самых популярных музыкальных синглов, а также стала одним из самых популярных синглов.


In [76]:

summarizer = SummarizationPipeline(model=model, tokenizer=tokenizer)
text = "Если бы в сериале «Нормальные люди» была сцена в дублинском подвальном клубе, то в нем играли бы ирландцы Fontaines D.C. Поэтичные постпанки сначала изображали народных героев и воспевали Дублин как «беременный город с католическим мозгом»; звучали они при этом как помесь The Clash и The Smiths. Как и Black Midi, группа собралась в стенах музыкального учебного заведения — на этот раз дублинского колледжа, а не английской школы, — но никакого намека на демонстративную виртуозность или заумность в их музыке нет.Между дебютным альбомом «Dogrel» (2019), названным в честь жанра уличной ирландской поэзии, и вторым «A Heroʼs Death» (июль 2020-го) прошло чуть более полутора лет, то есть совсем ничего по музыкальным меркам. Этого времени Fontaines D.С. хватило и на туры, пока еще была возможность куда-то ездить, и на новые песни, среди которых есть выдающаяся медитативная «I Donʼt Belong» о чувстве одиночества и неприкаянности. "
summary = summarizer(text, max_length=70, min_length=60, do_sample=False)

print(summary[0]['summary_text'])

В дублинском подвальном клубе была сцена в дублинском подвальном клубе, где играли ирландцы Fontaines D.C. Музыканты ирландцы Fontaines D.C. Музыканты ирландцы Fontaines D.C.


In [60]:
import torch

def predict():
    article = str(input())
    inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=100, truncation=True)
    return inputs.to('cuda')

inputs = predict()


model.to('cuda')

outputs = model.generate(
    inputs,
    max_length=40,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)


print(tokenizer.decode(outputs[0]))

Бристольская группа, совместившая находки американского нойз-рока (METZ, Pissed Jeans) не только с английским постпанком, но в первую очередь с яростью по поводу «Брекзита», более чем десятилетнего правления консерваторов в Великобритании, социального неравенства, расизма и ксенофобии. В альбоме «Brutalism» (2017) IDLES ярко выразили все эти тревоги молодых британцев в плакатно-лозунговой манере, после чего полюбились широкой публике, покорили британские чарты и быстро стали голосом рабочего класса. К нему участники группы — выпускники уважаемого бристольского вуза и владельцы местного андеграундного клуба — никакого отношения не имели, за что их не раз критиковали коллективы с более «правильным» происхождением. В Великобритании таких людей принято называть «классовыми туристами», но, как показывает история, часто именно им удается со стороны разглядеть и подчеркнуть самые важные проблемы.К третьему альбому «Ultra Mono» (2020) IDLES подошли в статусе одной из самых заметных британских 

In [62]:
import torch

def predict():
    article = str(input())
    inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=100, truncation=True)
    return inputs.to('cuda')

inputs = predict()

outputs = model.generate(
    inputs,
    max_length=40,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)


print(tokenizer.decode(outputs[0]))

Шведская панк-группа называет свое творчество «комментарием на тему провала роли мужчины в современном обществе», считает мужчин «корнем всех проблем» и призывает к свержению патриархата. Словом, с токсичной маскулинностью борются мужчины, которые сами выглядят как наглядная иллюстрация этого понятия.  Viagra Boys — чуть более легкая версия IDLES, но заметно превосходят бристольскую группу по энергетике живых выступлений. А еще они очень остры на язык, любят монотонные бас-партии и не боятся саксофона и других неожиданных для постпанка инструментов.
<pad>В Шведской панк-группа называет свое творчество «комментарием на тему провала роли мужчины в современном обществе» и призывает к сверж


In [63]:
import torch

def predict():
    article = str(input())
    inputs = tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=100, truncation=True)
    return inputs.to('cuda')

inputs = predict()

outputs = model.generate(
    inputs,
    max_length=40,
    min_length=5,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)


print(tokenizer.decode(outputs[0]))

Black Midi — группа выпускников престижной арт-школы для талантливых подростков BRIT School, которая с первых синглов поразила всех свободой музыкального мышления. В их музыке много мат-рока, краут-рока (недаром они выступали с легендарным экс-вокалистом Can Дамо Судзуки), шума, атональной импровизации, негармоничных созвучий и внезапной смены музыкальных размеров. При этом результат получается крайне захватывающим: сложности и навороты, которые сами по себе звучат довольно отталкивающе, в их песнях оказываются на своем месте и не кажутся излишними усложнениями.   В 2018 году Black Midi заметил британский продюсер Дэн Кэри — группа поездила по миру с турами, выступила на важных фестивалях и показала себя как впечатляющий проект с большим будущим, она явно должна была полюбиться сразу нескольким поколениям поклонников андеграундной музыки. Дебютный альбом Black Midi «Schlagenheim» вышел в начале 2019 года и действительно поразил не только многих музыкальных критиков, но и слушателей: гр